In [1]:
import os 
os.chdir(os.path.pardir)
# load data from file 
import numpy as np 
save_file_name = ['fea_seq.npy', 'last_observation_seq.npy', 'label_seq.npy', 'masking_seq.npy',
                   'delta_seq.npy', 'train_valid_test_split.npy']
save_folder = 'data/raw/predict-one-day-diff/pol-met-search'
saved_arrays = []
for file_name in save_file_name:
    saved_arrays.append(np.load(os.path.join(save_folder, file_name)))
[fea_seq, last_observation_seq, label_seq, masking_seq, delta_seq, train_valid_test_split] = saved_arrays


In [2]:
# train-test-split 
train_index = [k for k in range(train_valid_test_split[0])]
dev_index = [k for k in range(train_valid_test_split[0], 
                               train_valid_test_split[0] + train_valid_test_split[1])]
test_index = [k for k in range(train_valid_test_split[0] + train_valid_test_split[1],
              train_valid_test_split[0] + train_valid_test_split[1] + train_valid_test_split[2])]

In [3]:
def get_array_by_index_range(nparray_list, label_array, index_range):
    '''
    nparray_list: list of nparrays to select according to index range 
    label_array: select the labels from label array
    '''
    # get non-na index
    non_na_index = []
    for index in index_range:
        if not np.isnan(label_array[index]):
            non_na_index.append(index)
    
    return [k[non_na_index] for k in nparray_list], label_array[non_na_index].reshape(-1)

In [4]:
# split set to train, test and dev sets 
# train set
[fea_train, last_train], label_train =  get_array_by_index_range([fea_seq,last_observation_seq], label_seq, train_index)
# dev set 
[fea_dev, last_dev], label_dev =  get_array_by_index_range([fea_seq, last_observation_seq], label_seq, dev_index)
# test set 
[fea_test, last_test], label_test =  get_array_by_index_range([fea_seq, last_observation_seq], label_seq, test_index)

In [5]:
def normalize_feature(fea_train, array_list):
    """
    array_list: [fea_dev, fea_test, last_train, last_dev, last_test] to normalize 
    """
    train_mean = np.nanmean(fea_train, axis=0)
    train_std = np.nanstd(fea_train, axis=0)
    def norm_arr(nparr):
        return(nparr - train_mean)/train_std
    return (norm_arr(fea_train), [norm_arr(k) for k in array_list])

In [6]:
fea_train, [fea_dev, fea_test, last_train, last_dev, last_test] = normalize_feature(fea_train,
                                                                                   [fea_dev, fea_test, 
                                                                                    last_train, last_dev,
                                                                                    last_test])

In [7]:
# record mean after normalization 
x_mean_aft_nor = np.nanmean(fea_train, axis=0)

In [8]:
# control experiment using last observed value for missing data imputation 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from torch.autograd import Variable, grad
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math

In [9]:
fea_train.shape

(664, 7, 52)

In [15]:
def train_mfn(X_train, y_train, X_valid, y_valid, X_test, y_test, configs):
#     p = np.random.permutation(X_train.shape[0])
    # no shuffle, keep original order 
    # swap axes for back propagation 
    def swap_axes(nparr):
        return nparr.swapaxes(0,1)
    X_train = swap_axes(X_train)
    X_valid = swap_axes(X_valid)
    X_test = swap_axes(X_test)
    
    # model parameters 
    input_size = X_train.shape[2]
    h = 128
    t = X_train.shape[0]
    output_dim = 1
    dropout = 0.5

#     d = X_train.shape[2]
#     h = 128
#     t = X_train.shape[0]
#     output_dim = 1
#     dropout = 0.5

    [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig] = configs

    
    #model = EFLSTM(d,h,output_dim,dropout)
    model = MFN(config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig)

    optimizer = optim.Adam(model.parameters(),lr=config["lr"])
    #optimizer = optim.SGD(model.parameters(),lr=config["lr"],momentum=config["momentum"])

    # optimizer = optim.SGD([
    #                 {'params':model.lstm_l.parameters(), 'lr':config["lr"]},
    #                 {'params':model.classifier.parameters(), 'lr':config["lr"]}
    #             ], momentum=0.9)

    criterion = nn.MSELoss()
    device = torch.device('cpu')
    model = model.to(device)
    criterion = criterion.to(device)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, verbose=True)
    
#     criterion = nn.L1Loss()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)
#     criterion = criterion.to(device)
#     scheduler = ReduceLROnPlateau(optimizer,mode='min',patience=100,factor=0.5,verbose=True)

    def train(model, batchsize, X_train, y_train, optimizer, criterion):
        epoch_loss = 0
        model.train()
        total_n = X_train.shape[1]
        num_batches = math.ceil(total_n / batchsize)
        for batch in range(num_batches):
            start = batch*batchsize
            end = (batch+1)*batchsize
            optimizer.zero_grad()
            batch_X = torch.Tensor(X_train[:,start:end])
            batch_y = torch.Tensor(y_train[start:end])
            predictions = model.forward(batch_X).squeeze(1)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / num_batches

    def evaluate(model, X_valid, y_valid, criterion):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_valid)
            batch_y = torch.Tensor(y_valid)
            predictions = model.forward(batch_X).squeeze(1)
            epoch_loss = criterion(predictions, batch_y).item()
        return epoch_loss

    def predict(model, X_test):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_test)
            predictions = model.forward(batch_X).squeeze(1)
            predictions = predictions.cpu().data.numpy()
        return predictions

    best_valid = 999999.0
    rand = random.randint(0,100000)
    print('epoch train_loss valid_loss')
    for epoch in range(config["num_epochs"]):
        train_loss = train(model, config["batchsize"], X_train, y_train, optimizer, criterion)
        valid_loss = evaluate(model, X_valid, y_valid, criterion)
        scheduler.step(valid_loss)
        if valid_loss <= best_valid:
            # save model
            best_valid = valid_loss
            print(epoch, train_loss, valid_loss, 'saving model')
            torch.save(model, 'models/temp_models/mfn_%d.pt' %rand)
        else:
            print(epoch, train_loss, valid_loss)

#     print 'model number is:', rand
    model = torch.load('models/temp_models/mfn_%d.pt' %rand)

    predictions = predict(model, X_test)
    mae = np.mean(np.absolute(predictions-y_test))
    print("mae: ", mae)
    mse = np.mean((predictions - y_test)**2)
    print("mse: ", mse)

In [16]:
class MFN(nn.Module):
    def __init__(self,config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig):
        super(MFN, self).__init__()
        [self.d_l,self.d_a] = config["input_dims"]
        [self.dh_l,self.dh_a] = config["h_dims"]
        total_h_dim = self.dh_l+self.dh_a
        
        self.mem_dim = config["memsize"]
        window_dim = config["windowsize"]
        output_dim = 1
        attInShape = total_h_dim*window_dim
        gammaInShape = attInShape+self.mem_dim
        final_out = total_h_dim+self.mem_dim
        h_att1 = NN1Config["shapes"]
        h_att2 = NN2Config["shapes"]
        h_gamma1 = gamma1Config["shapes"]
        h_gamma2 = gamma2Config["shapes"]
        h_out = outConfig["shapes"]
        att1_dropout = NN1Config["drop"]
        att2_dropout = NN2Config["drop"]
        gamma1_dropout = gamma1Config["drop"]
        gamma2_dropout = gamma2Config["drop"]
        out_dropout = outConfig["drop"]

        self.lstm_l = nn.GRUCell(self.d_l, self.dh_l)
        self.lstm_a = nn.GRUCell(self.d_a, self.dh_a)

        self.att1_fc1 = nn.Linear(attInShape, h_att1)
        self.att1_fc2 = nn.Linear(h_att1, attInShape)
        self.att1_dropout = nn.Dropout(att1_dropout)

        self.att2_fc1 = nn.Linear(attInShape, h_att2)
        self.att2_fc2 = nn.Linear(h_att2, self.mem_dim)
        self.att2_dropout = nn.Dropout(att2_dropout)

        self.gamma1_fc1 = nn.Linear(gammaInShape, h_gamma1)
        self.gamma1_fc2 = nn.Linear(h_gamma1, self.mem_dim)
        self.gamma1_dropout = nn.Dropout(gamma1_dropout)

        self.gamma2_fc1 = nn.Linear(gammaInShape, h_gamma2)
        self.gamma2_fc2 = nn.Linear(h_gamma2, self.mem_dim)
        self.gamma2_dropout = nn.Dropout(gamma2_dropout)

        self.out_fc1 = nn.Linear(final_out, h_out)
        self.out_fc2 = nn.Linear(h_out, output_dim)
        self.out_dropout = nn.Dropout(out_dropout)

    def forward(self,x):
        x_l = x[:,:,:self.d_l]
        x_a = x[:,:,self.d_l:self.d_l+self.d_a]
        # x is t x n x d
        n = x.shape[1]
        t = x.shape[0]
        self.h_l = torch.zeros(n, self.dh_l)
        self.h_a = torch.zeros(n, self.dh_a)

        self.c_l = torch.zeros(n, self.dh_l)
        self.c_a = torch.zeros(n, self.dh_a)
        
        self.mem = torch.zeros(n, self.mem_dim)
        all_h_ls = []
        all_h_as = []

        all_c_ls = []
        all_c_as = []

        all_mems = []
        for i in range(t):
            # prev time step
            prev_h_l = self.h_l
            prev_h_a = self.h_a

            # curr time step
            new_h_l = self.lstm_l(x_l[i], self.h_l)
            new_h_a = self.lstm_a(x_a[i], self.h_a)
   
            # concatenate
            prev_cs = torch.cat([prev_h_l,prev_h_a], dim=1)
            new_cs = torch.cat([new_h_l,new_h_a], dim=1)
            
            cStar = torch.cat([prev_cs,new_cs], dim=1)
            attention = F.softmax(self.att1_fc2(self.att1_dropout(F.relu(self.att1_fc1(cStar)))),dim=1)
            attended = attention*cStar
            
            cHat = F.tanh(self.att2_fc2(self.att2_dropout(F.relu(self.att2_fc1(attended)))))
            
            both = torch.cat([attended,self.mem], dim=1)
            gamma1 = F.sigmoid(self.gamma1_fc2(self.gamma1_dropout(F.relu(self.gamma1_fc1(both)))))
            gamma2 = F.sigmoid(self.gamma2_fc2(self.gamma2_dropout(F.relu(self.gamma2_fc1(both)))))
            
            self.mem = gamma1*self.mem + gamma2*cHat
            all_mems.append(self.mem)
            # update
            self.h_l = new_h_l
            self.h_a = new_h_a

            all_h_ls.append(self.h_l)
            all_h_as.append(self.h_a)

        # last hidden layer last_hs is n x h
        last_h_l = all_h_ls[-1]
        last_h_a = all_h_as[-1]

        last_mem = all_mems[-1]
        last_hs = torch.cat([last_h_l,last_h_a,last_mem], dim=1)
        output = self.out_fc2(self.out_dropout(F.relu(self.out_fc1(last_hs))))
        return output

In [17]:
config = dict()
config["input_dims"] = [4, 47]
hl = 256
ha = 256
drop = 0.7
config["h_dims"] = [hl, ha]
config["memsize"] = hl
config["windowsize"] = 2
config["batchsize"] = hl
config["num_epochs"] = 80
config["lr"] = 0.0005
NN1Config = dict()
NN1Config["shapes"] = hl
NN1Config["drop"] = drop
NN2Config = dict()
NN2Config["shapes"] = 32
NN2Config["drop"] = drop
gamma1Config = dict()
gamma1Config["shapes"] = hl
gamma1Config["drop"] = drop
gamma2Config = dict()
gamma2Config["shapes"] = hl 
gamma2Config["drop"] = drop
outConfig = dict() 
outConfig["shapes"] = hl
outConfig["drop"] = drop
configs = [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig]

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(last_train[:,:,1:], label_train, last_dev[:,:,1:], label_dev, last_test[:,:,1:], label_test, configs)

epoch train_loss valid_loss
0 40.60454813639323 33.560943603515625 saving model
1 40.45938491821289 33.44845199584961 saving model
2 40.39717356363932 33.322845458984375 saving model
3 40.2489865620931 33.1762809753418 saving model
4 40.09260940551758 33.000648498535156 saving model
5 39.98527399698893 32.798492431640625 saving model
6 39.834634145100914 32.57582473754883 saving model
7 39.73832321166992 32.35267639160156 saving model
8 39.538394927978516 32.13829803466797 saving model
9 39.270407358805336 31.897539138793945 saving model
10 38.85156504313151 31.603099822998047 saving model
11 38.20111338297526 31.230321884155273 saving model
12 37.498269399007164 30.785438537597656 saving model
13 36.54711659749349 30.396337509155273 saving model
14 35.46378644307455 30.359203338623047 saving model
15 34.40589841206869 30.790096282958984
16 33.484090169270836 30.6092472076416
17 32.17392794291178 29.768112182617188 saving model
18 31.05070622762044 29.419456481933594 saving model
19 29

In [18]:
class MFN(nn.Module):
    def __init__(self,config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig):
        super(MFN, self).__init__()
        [self.d_l,self.d_a] = config["input_dims"]
        [self.dh_l,self.dh_a] = config["h_dims"]
        total_h_dim = self.dh_l+self.dh_a
        
        self.mem_dim = config["memsize"]
        window_dim = config["windowsize"]
        output_dim = 1
        attInShape = total_h_dim*window_dim
        gammaInShape = attInShape+self.mem_dim
        final_out = total_h_dim+self.mem_dim
        h_att1 = NN1Config["shapes"]
        h_att2 = NN2Config["shapes"]
        h_gamma1 = gamma1Config["shapes"]
        h_gamma2 = gamma2Config["shapes"]
        h_out = outConfig["shapes"]
        att1_dropout = NN1Config["drop"]
        att2_dropout = NN2Config["drop"]
        gamma1_dropout = gamma1Config["drop"]
        gamma2_dropout = gamma2Config["drop"]
        out_dropout = outConfig["drop"]

        self.lstm_l = nn.LSTMCell(self.d_l, self.dh_l)
        self.lstm_a = nn.LSTMCell(self.d_a, self.dh_a)

        self.att1_fc1 = nn.Linear(attInShape, h_att1)
        self.att1_fc2 = nn.Linear(h_att1, attInShape)
        self.att1_dropout = nn.Dropout(att1_dropout)

        self.att2_fc1 = nn.Linear(attInShape, h_att2)
        self.att2_fc2 = nn.Linear(h_att2, self.mem_dim)
        self.att2_dropout = nn.Dropout(att2_dropout)

        self.gamma1_fc1 = nn.Linear(gammaInShape, h_gamma1)
        self.gamma1_fc2 = nn.Linear(h_gamma1, self.mem_dim)
        self.gamma1_dropout = nn.Dropout(gamma1_dropout)

        self.gamma2_fc1 = nn.Linear(gammaInShape, h_gamma2)
        self.gamma2_fc2 = nn.Linear(h_gamma2, self.mem_dim)
        self.gamma2_dropout = nn.Dropout(gamma2_dropout)

        self.out_fc1 = nn.Linear(final_out, h_out)
        self.out_fc2 = nn.Linear(h_out, output_dim)
        self.out_dropout = nn.Dropout(out_dropout)

    def forward(self,x):
        x_l = x[:,:,:self.d_l]
        x_a = x[:,:,self.d_l:self.d_l+self.d_a]
        # x is t x n x d
        n = x.shape[1]
        t = x.shape[0]
        self.h_l = torch.zeros(n, self.dh_l)
        self.h_a = torch.zeros(n, self.dh_a)

        self.c_l = torch.zeros(n, self.dh_l)
        self.c_a = torch.zeros(n, self.dh_a)
        
        self.mem = torch.zeros(n, self.mem_dim)
        all_h_ls = []
        all_h_as = []

        all_c_ls = []
        all_c_as = []

        all_mems = []
        for i in range(t):
            # prev time step
            prev_c_l = self.c_l
            prev_c_a = self.c_a

            # curr time step
            new_h_l, new_c_l = self.lstm_l(x_l[i], (self.h_l, self.c_l))
            new_h_a, new_c_a = self.lstm_a(x_a[i], (self.h_a, self.c_a))
   
            # concatenate
            prev_cs = torch.cat([prev_c_l,prev_c_a], dim=1)
            new_cs = torch.cat([new_c_l,new_c_a], dim=1)
            
            cStar = torch.cat([prev_cs,new_cs], dim=1)
            attention = F.softmax(self.att1_fc2(self.att1_dropout(F.relu(self.att1_fc1(cStar)))),dim=1)
            attended = attention*cStar
            
            cHat = F.tanh(self.att2_fc2(self.att2_dropout(F.relu(self.att2_fc1(attended)))))
            
            both = torch.cat([attended,self.mem], dim=1)
            gamma1 = F.sigmoid(self.gamma1_fc2(self.gamma1_dropout(F.relu(self.gamma1_fc1(both)))))
            gamma2 = F.sigmoid(self.gamma2_fc2(self.gamma2_dropout(F.relu(self.gamma2_fc1(both)))))
            
            self.mem = gamma1*self.mem + gamma2*cHat
            all_mems.append(self.mem)
            # update
            self.h_l, self.c_l = new_h_l, new_c_l
            self.h_a, self.c_a = new_h_a, new_c_a

            all_h_ls.append(self.h_l)
            all_h_as.append(self.h_a)
 
            all_c_ls.append(self.c_l)
            all_c_as.append(self.c_a)

        # last hidden layer last_hs is n x h
        last_h_l = all_h_ls[-1]
        last_h_a = all_h_as[-1]

        last_mem = all_mems[-1]
        last_hs = torch.cat([last_h_l,last_h_a,last_mem], dim=1)
        output = self.out_fc2(self.out_dropout(F.relu(self.out_fc1(last_hs))))
        return output

In [19]:
config = dict()
config["input_dims"] = [4, 47]
hl = 256
ha = 256
drop = 0.7
config["h_dims"] = [hl, ha]
config["memsize"] = hl
config["windowsize"] = 2
config["batchsize"] = hl
config["num_epochs"] = 80
config["lr"] = 0.0005
NN1Config = dict()
NN1Config["shapes"] = hl
NN1Config["drop"] = drop
NN2Config = dict()
NN2Config["shapes"] = 32
NN2Config["drop"] = drop
gamma1Config = dict()
gamma1Config["shapes"] = hl
gamma1Config["drop"] = drop
gamma2Config = dict()
gamma2Config["shapes"] = hl 
gamma2Config["drop"] = drop
outConfig = dict() 
outConfig["shapes"] = hl
outConfig["drop"] = drop
configs = [config,NN1Config,NN2Config,gamma1Config,gamma2Config,outConfig]

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_mfn(last_train[:,:,1:], label_train, last_dev[:,:,1:], label_dev, last_test[:,:,1:], label_test, configs)

epoch train_loss valid_loss
0 40.61702982584635 33.58317184448242 saving model
1 40.529825846354164 33.52313232421875 saving model
2 40.46124267578125 33.4550895690918 saving model
3 40.396976470947266 33.37479019165039 saving model
4 40.44697570800781 33.283809661865234 saving model
5 40.35490417480469 33.17273712158203 saving model
6 40.152960459391274 33.02576446533203 saving model
7 40.03765741984049 32.84747314453125 saving model
8 40.03054682413737 32.6578254699707 saving model
9 39.78166198730469 32.43970489501953 saving model
10 39.5495719909668 32.176422119140625 saving model
11 39.222390492757164 31.82850456237793 saving model
12 38.681479136149086 31.36661720275879 saving model
13 38.073123931884766 30.80330467224121 saving model
14 37.13399124145508 30.26140785217285 saving model
15 35.839656829833984 29.96523094177246 saving model
16 35.06632296244303 30.117525100708008
17 34.04050064086914 29.39242172241211 saving model
18 32.36841011047363 28.494014739990234 saving model